In [ ]:
!pip install -Uqq nixtla
!pip install -q dask[dataframe]
!pip install -q neuralforecast

In [ ]:
import time
import requests
import pandas as pd
import numpy as np

from nixtla import NixtlaClient

from utilsforecast.losses import mae, smape, mse
from utilsforecast.evaluation import evaluate

from neuralforecast.core import NeuralForecast
from neuralforecast.models import NHITS, LSTM
from neuralforecast.losses.pytorch import DistributionLoss

from utilsforecast.plotting import plot_series
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt


nixtla_client = NixtlaClient(
    # defaults to os.environ.get("NIXTLA_API_KEY")
    api_key = 'nixak-u0uuMKxJQLH7NljBpZRjckwKuENxptz6beRKSaVPvjPheR4P5H9J3Ju9rK47rGoN4qeUqlxgoqICfJ1O'
)

df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/refs/heads/main/datasets/pjm_in_zone.csv')
df['ds'] = pd.to_datetime(df['ds'])


df.groupby('unique_id').head(2)

In [ ]:
def generate_inputs(df, days=73):

    test_df = df.groupby('unique_id').tail(24 * days)
    train_df = df.groupby('unique_id').apply(lambda group: group.iloc[:-1 * (24 * days)]).reset_index(drop=True)


    #create list of 73 days true values for each unique_id
    Y_df = [
    test_df.groupby('unique_id', group_keys=False).apply(lambda group: group.iloc[i:i+24]).reset_index(drop=True)
    for i in range(0, 24 * days, 24)
    ]

    p = 1

    #create paired inputs
    inputs_df = [Y_df[i] for i in range(len(Y_df) - p)]

    inputs_df.insert(0, train_df.groupby('unique_id', group_keys=False).apply(lambda group: group.iloc[-24:]))


    return Y_df, inputs_df, train_df, test_df


In [ ]:
Y_df, inputs_df, train_df, test_df = generate_inputs(df)

#Models

In [ ]:
models = [NHITS(
                h=24,
                input_size=24,
                scaler_type='robust',
                batch_size=16,
                valid_batch_size=8,
                max_steps=3000
          ),

          LSTM(
              h=24,
              input_size=24,
              scaler_type='robust',
              encoder_n_layers=1,
              encoder_hidden_size=64,
              context_size=128,
              decoder_hidden_size=128,
              decoder_layers=5,
              learning_rate=1e-4,
              encoder_dropout=0.1,
              max_steps=3000
          )
]

nf = NeuralForecast(models=models, freq='h')
nf.fit(df=train_df)

In [ ]:
nf.save(
    path='./checkpoint/models/',
    model_index=None,
    overwrite=True,
    save_dataset=True
)

In [ ]:
def funcs_test(inputs_df, Y_df):

  preds = [nf.predict(inputs_df[i]) for i in range(len(Y_df))]
  y_preds_df = [pd.merge(Y_df[i], preds[i], 'left', ['unique_id', 'ds']) for i in range(len(Y_df))]

  evals = []

  for i in range(len(y_preds_df)):
      evaluation = evaluate(
                  y_preds_df[i],
                  metrics=[mse, mae, smape],
                  models=['NHITS', 'LSTM'],
                  target_col='y',
                  id_col='unique_id'
                )

      evals.append(evaluation)

  return evals, preds, y_preds_df

In [ ]:
evaluations, preds, y_preds_df = funcs_test(inputs_df, Y_df)

In [ ]:
preds_concat= pd.concat(preds, ignore_index=True)
Y_df_concat = pd.concat(Y_df, ignore_index=True)

#TimeGPT

In [ ]:

def TimegptForecast():
  full_forecast = []
  training_data = train_df.copy()

  for i in range(0, 24 * 73, 24):

      fcst_df = nixtla_client.forecast(
          df=training_data,
          h=24,
          finetune_steps=10,
          finetune_loss='mae',
          model='timegpt-1',
          time_col='ds',
          target_col='y',
          id_col='unique_id'
      )

      #1 day from train set + 1 day from test -> 2 days from test -> next 2 days from test
      training_data = pd.concat([training_data, test_df.groupby('unique_id', group_keys=False).apply(lambda group: group.iloc[i:i+24])], ignore_index=True)

      full_forecast.append(fcst_df)

  return full_forecast


In [ ]:
forecasts = TimegptForecast()

In [ ]:
forecasts = pd.concat(forecasts, ignore_index=True)

In [ ]:
all_res = pd.merge(preds_concat, forecasts, 'left', ['unique_id', 'ds'])

In [ ]:

fig, axes = plt.subplots(nrows=5, ncols=1, figsize=(64, 32))


nixtla_client.plot(Y_df_concat.groupby('unique_id').head(876),
                   all_res.groupby('unique_id').head(876),
                   models=['TimeGPT', 'LSTM', 'NHITS'], time_col='ds',
                   target_col='y', ax=axes)

In [ ]:
Y_df_concat = pd.concat(Y_df, ignore_index=True)

In [ ]:
eval_df = pd.merge(Y_df_concat, all_res, 'left', ['unique_id', 'ds'])
eval_df.to_csv('nixtla_preds.csv', index=False)

In [ ]:
evaluation = evaluate(
    eval_df,
    metrics=[mse, mae, smape],
    models=['NHITS', 'LSTM', 'TimeGPT'],
    target_col='y',
    id_col='unique_id'
)


In [ ]:
pd.set_option('display.float_format', '{:.6f}'.format)
evaluation.sort_values(['unique_id', 'metric']).groupby('unique_id', group_keys=False).head()

In [ ]:
nhits_mean = evaluation.groupby('metric')['NHITS'].mean().apply(lambda x: f"{x:.6f}")
timegpt_mean = evaluation.groupby('metric')['TimeGPT'].mean().apply(lambda x: f"{x:.6f}")
lstm_mean = evaluation.groupby('metric')['LSTM'].mean().apply(lambda x: f"{x:.6f}")

formatted_df = pd.DataFrame({
    "NHITS": nhits_mean,
    "LSTM": lstm_mean,
    "TimeGPT": timegpt_mean,
})

formatted_df

In [ ]:
all_res.to_csv("nixtla_preds.csv")